In [3]:
import polars as pl

In [28]:
spy_df = pl.scan_csv("SPY_close_price_5Y.csv", try_parse_dates=True)

In [33]:
finalspy_df = spy_df.with_columns([
    pl.col("Close").rolling_mean(window_size=50, min_samples=50).alias("ma_50"),
    pl.col("Close").rolling_mean(window_size=200, min_samples=200).alias("ma_200")
]).with_columns([
    ((pl.col("ma_50") > pl.col("ma_200")) & 
     (pl.col("ma_50").shift(1) <= pl.col("ma_200").shift(1))).cast(pl.Int32).alias("gc")
]).filter(pl.col("gc") == 1).collect()

In [34]:
finalspy_df

Date,Close,ma_50,ma_200,gc
date,f64,f64,f64,i32
2023-02-02,416.78,394.3504,394.25025,1
2025-07-01,617.65,583.0978,582.04155,1
